In [572]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [573]:
df=pd.read_csv('labeled_data2.csv')

In [574]:
df.columns

Index(['Unnamed: 0', 'src_ip', 'dst_ip', 'src_port', 'dst_port', 'protocol',
       'timestamp', 'flow_duration', 'flow_byts_s', 'flow_pkts_s',
       'fwd_pkts_s', 'bwd_pkts_s', 'tot_fwd_pkts', 'tot_bwd_pkts',
       'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'pkt_len_max', 'pkt_len_min', 'pkt_len_mean',
       'pkt_len_std', 'pkt_len_var', 'fwd_header_len', 'bwd_header_len',
       'fwd_seg_size_min', 'fwd_act_data_pkts', 'flow_iat_mean',
       'flow_iat_max', 'flow_iat_min', 'flow_iat_std', 'fwd_iat_tot',
       'fwd_iat_max', 'fwd_iat_min', 'fwd_iat_mean', 'fwd_iat_std',
       'bwd_iat_tot', 'bwd_iat_max', 'bwd_iat_min', 'bwd_iat_mean',
       'bwd_iat_std', 'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags',
       'bwd_urg_flags', 'fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt',
       'psh_flag_cnt', 'ack_flag_c

In [575]:
df.head()

,Unnamed: 0,src_ip,dst_ip,src_port,dst_port,protocol,timestamp,flow_duration,flow_byts_s,flow_pkts_s,...,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts,Label
0,0,10.215.173.1,157.240.16.32,48924,443,6,2021-09-15 15:33:24,2525190.0,2094.495860,8.316206,...,0.000000,0.0,126.400000,365.909091,0,10,11,1264,4025,benign
1,1,10.215.173.1,10.215.173.2,16658,53,17,2021-09-15 15:33:24,43626.0,4148.902031,45.844221,...,0.000000,0.0,68.000000,113.000000,0,1,1,68,113,benign
2,2,10.215.173.1,157.240.16.53,37292,5222,6,2021-09-15 15:33:24,2504540.0,735.065122,9.183323,...,71225.071225,0.0,70.454545,88.833333,0,11,12,775,1066,benign
3,3,192.168.101.162,172.217.160.162,39604,443,17,2021-09-15 15:33:24,270572.0,1075.499313,18.479370,...,0.000000,0.0,61.666667,53.000000,0,3,2,185,106,benign
4,4,10.215.173.1,192.168.101.238,44848,53,17,2021-09-15 15:33:24,230191.0,981.793380,8.688437,...,0.000000,0.0,77.000000,149.000000,0,1,1,77,149,benign


In [576]:
data = [df["ack_flag_cnt"], df["init_fwd_win_byts"],df["fwd_seg_size_min"], df["fwd_iat_tot"],df["flow_duration"], df["Label"]]

In [577]:
headers = ["ack", "init","seg","iat","flow","label"]

In [578]:
df1 = pd.concat(data, axis=1, keys=headers)

In [579]:
from sklearn.preprocessing import LabelEncoder

In [580]:
Encoder = LabelEncoder()
le = Encoder.fit(df1["label"])
df1["label"] = le.transform(df1["label"])

In [581]:
df1
# 1 = Benign
# 0 = Attack

,ack,init,seg,iat,flow,label
0,0,65535,20,582767.0,2525190.0,1
1,0,0,8,0.0,43626.0,1
2,0,65535,20,862174.0,2504540.0,1
3,0,0,8,270572.0,270572.0,1
4,0,0,8,0.0,230191.0,1
...,...,...,...,...,...,...
54961,0,0,8,0.0,134195.0,1
54962,0,0,8,0.0,136412.0,1
54963,0,0,8,238581.0,296628.0,1
54964,0,0,8,0.0,163704.0,1


In [582]:
X = df1.drop('label',axis=1)
Y = df1['label']

In [583]:
X

,ack,init,seg,iat,flow
0,0,65535,20,582767.0,2525190.0
1,0,0,8,0.0,43626.0
2,0,65535,20,862174.0,2504540.0
3,0,0,8,270572.0,270572.0
4,0,0,8,0.0,230191.0
...,...,...,...,...,...
54961,0,0,8,0.0,134195.0
54962,0,0,8,0.0,136412.0
54963,0,0,8,238581.0,296628.0
54964,0,0,8,0.0,163704.0


In [584]:
Y

0        1
1        1
2        1
3        1
4        1
        ..
54961    1
54962    1
54963    1
54964    1
54965    1
Name: label, Length: 54966, dtype: int32

In [585]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier 
import lightgbm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [586]:
rf_temp = 0
rf_counter = 1
ada_temp = 0
ada_counter = 1
xgb_temp = 0
xgb_counter = 1
lgbm_temp = 0
lgbm_counter = 1

# 5 folds

In [587]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [588]:
ada_lst_accu_stratified = []
ada_lst_f1_stratified = []
ada_lst_prec_stratified = []
ada_lst_recall_stratified = []
ada_lst_aucroc_stratified = []

xgb_lst_accu_stratified = []
xgb_lst_f1_stratified = []
xgb_lst_prec_stratified = []
xgb_lst_recall_stratified = []
xgb_lst_aucroc_stratified = []

rf_lst_accu_stratified = []
rf_lst_f1_stratified = []
rf_lst_prec_stratified = []
rf_lst_recall_stratified = []
rf_lst_aucroc_stratified = []

lgbm_lst_accu_stratified = []
lgbm_lst_f1_stratified = []
lgbm_lst_prec_stratified = []
lgbm_lst_recall_stratified = []
lgbm_lst_aucroc_stratified = []

In [589]:
xgb = XGBClassifier(scale_pos_weight=1)
ada = AdaBoostClassifier()
rf = RandomForestClassifier()
lgbm = lightgbm.LGBMClassifier()

In [590]:
    xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=1, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='gpu_hist', use_label_encoder=True,
              validate_parameters=1, verbosity=0
                       )
    
    ada = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                           n_estimators=50, random_state=1)
    rf = RandomForestClassifier(bootstrap=True,ccp_alpha=0.0,
                               class_weight=None,criterion='gini',
                               max_depth=None,max_features='auto',
                               max_leaf_nodes=None, max_samples=None,
                               min_impurity_decrease=0.0,min_samples_leaf=1, min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=100,
                               n_jobs=-1,oob_score=False,random_state=1,verbose=0,
                               warm_start=False)
    lgbm = lightgbm.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                                  importance_type='split', learning_rate=0.1,
                                   max_depth=-1, min_child_samples=20, min_child_weight=0.001,
                                   min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
                                   objective=None, random_state=1, reg_alpha=0.0, reg_lambda=0.0,
                                   silent=True, subsample=1.0, subsample_for_bin=200000,
                                   subsample_freq=0)

In [591]:
%%time
for train_index, test_index in skf.split(X, Y):
    print(len(test_index))
    
    x_train_fold, x_test_fold = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train_fold, y_test_fold = Y[train_index], Y[test_index]
    oversample = SMOTE()
    x_train, y_train = oversample.fit_resample(x_train_fold, y_train_fold)
    

    
    xgb_clf = xgb.fit(x_train,y_train)
    ada_clf = ada.fit(x_train,y_train)
    rf_clf = rf.fit(x_train,y_train)
    lgbm_clf = lgbm.fit(x_train,y_train)
    
    y_pred = ada.predict(x_test_fold)
    y_true = y_test_fold.tolist()
    ada_acc = accuracy_score(y_true,y_pred)
    ada_f1 = f1_score(y_true,y_pred)
    ada_precision = precision_score(y_true,y_pred)
    ada_recall = recall_score(y_true,y_pred)
    ada_aucroc = roc_auc_score(y_true,y_pred)
    ada_lst_accu_stratified.append(ada_acc)
    ada_lst_f1_stratified.append(ada_f1)
    ada_lst_prec_stratified.append(ada_precision)
    ada_lst_recall_stratified.append(ada_recall)
    ada_lst_aucroc_stratified.append(ada_aucroc)
    
    y_pred = xgb.predict(x_test_fold)
    xgb_acc = accuracy_score(y_true,y_pred)
    xgb_f1 = f1_score(y_true,y_pred)
    xgb_precision = precision_score(y_true,y_pred)
    xgb_recall = recall_score(y_true,y_pred)
    xgb_aucroc = roc_auc_score(y_true,y_pred)
    xgb_lst_accu_stratified.append(xgb_acc)
    xgb_lst_f1_stratified.append(xgb_f1)
    xgb_lst_prec_stratified.append(xgb_precision)
    xgb_lst_recall_stratified.append(xgb_recall)
    xgb_lst_aucroc_stratified.append(xgb_aucroc)
    
    y_pred = lgbm.predict(x_test_fold)
    lgbm_acc = accuracy_score(y_true,y_pred)
    lgbm_f1 = f1_score(y_true,y_pred)
    lgbm_precision = precision_score(y_true,y_pred)
    lgbm_recall = recall_score(y_true,y_pred)
    lgbm_aucroc = roc_auc_score(y_true,y_pred)
    lgbm_lst_accu_stratified.append(lgbm_acc)
    lgbm_lst_f1_stratified.append(lgbm_f1)
    lgbm_lst_prec_stratified.append(lgbm_precision)
    lgbm_lst_recall_stratified.append(lgbm_recall)
    lgbm_lst_aucroc_stratified.append(lgbm_aucroc)
    
    y_pred = rf.predict(x_test_fold)
    rf_acc = accuracy_score(y_true,y_pred)
    rf_f1 = f1_score(y_true,y_pred)
    rf_precision = precision_score(y_true,y_pred)
    rf_recall = recall_score(y_true,y_pred)
    rf_aucroc = roc_auc_score(y_true,y_pred)
    rf_lst_accu_stratified.append(rf_acc)
    rf_lst_f1_stratified.append(rf_f1)
    rf_lst_prec_stratified.append(rf_precision)
    rf_lst_recall_stratified.append(rf_recall)
    rf_lst_aucroc_stratified.append(rf_aucroc)

    if rf_f1 > rf_temp:
        rf_temp = rf_f1
        filename = str(rf_counter)+"rf_model.pkl"
        pickle.dump(rf_clf,open(filename,'wb'))
        print("rf "+str(rf_counter))
    rf_counter = rf_counter+1
        
    if ada_f1 > ada_temp:
        ada_temp = ada_f1
        filename = str(ada_counter)+"ada_model.pkl"
        pickle.dump(ada_clf,open(filename,'wb'))
        print("ada " + str(ada_counter))
    ada_counter = ada_counter+1
    
    if xgb_f1 > xgb_temp:
        xgb_temp = xgb_f1
        filename = str(xgb_counter)+"xgb_model.pkl"
        pickle.dump(xgb_clf,open(filename,'wb'))
        print("xgb "+str(xgb_counter))
    xgb_counter = xgb_counter+1
        
    if lgbm_f1 > lgbm_temp:
        lgbm_temp = lgbm_f1
        filename = str(lgbm_counter)+"lgbm_model.pkl"
        pickle.dump(lgbm_clf,open(filename,'wb'))
        print("lgbm "+str(lgbm_counter))
    lgbm_counter = lgbm_counter+1
    

10994


D:\anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


rf 1
ada 1
xgb 1
lgbm 1
10993


D:\anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


rf 2
ada 2
xgb 2
lgbm 2
10993


...

ada 3
10993


D:\anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


ada 4
10993


D:\anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


lgbm 5
Wall time: 22.4 s


# rf

In [592]:
print ("Average accuracy => "+ str(sum(rf_lst_accu_stratified)/len(rf_lst_accu_stratified)))

Average accuracy => 0.9943055763353865


In [593]:
print ("Average F1 score => "+ str(sum(rf_lst_f1_stratified)/len(rf_lst_f1_stratified)))

Average F1 score => 0.8982848269290334


In [594]:
print ("Average precision => "+ str(sum(rf_lst_prec_stratified)/len(rf_lst_prec_stratified)))

Average precision => 0.8262563161596459


In [595]:
print ("Average recall => "+ str(sum(rf_lst_recall_stratified)/len(rf_lst_recall_stratified)))

Average recall => 0.9843111337061515


In [596]:
print ("Average AUC ROC score => "+ str(sum(rf_lst_aucroc_stratified)/len(rf_lst_aucroc_stratified)))

Average AUC ROC score => 0.9894391432969865


## ada

In [597]:
print ("Average accuracy => "+ str(sum(ada_lst_accu_stratified)/len(ada_lst_accu_stratified)))

Average accuracy => 0.9890114090331616


In [598]:
print ("Average F1 score => "+ str(sum(ada_lst_f1_stratified)/len(ada_lst_f1_stratified)))

Average F1 score => 0.8219132540425967


In [599]:
print ("Average precision => "+ str(sum(ada_lst_prec_stratified)/len(ada_lst_prec_stratified)))

Average precision => 0.7017431006519156


In [600]:
print ("Average recall => "+ str(sum(ada_lst_recall_stratified)/len(ada_lst_recall_stratified)))

Average recall => 0.9921657346212506


In [601]:
print ("Average AUC ROC score => "+ str(sum(ada_lst_aucroc_stratified)/len(ada_lst_aucroc_stratified)))

Average AUC ROC score => 0.9905473316248841


## xgb

In [602]:
print ("Average accuracy => "+ str(sum(xgb_lst_accu_stratified)/len(xgb_lst_accu_stratified)))

Average accuracy => 0.9919041010452663


In [603]:
print ("Average F1 score => "+ str(sum(xgb_lst_f1_stratified)/len(xgb_lst_f1_stratified)))

Average F1 score => 0.8606993353462272


In [604]:
print ("Average precision => "+ str(sum(xgb_lst_prec_stratified)/len(xgb_lst_prec_stratified)))

Average precision => 0.7684701195042392


In [605]:
print ("Average recall => "+ str(sum(xgb_lst_recall_stratified)/len(xgb_lst_recall_stratified)))

Average recall => 0.9786171835282156


In [606]:
print ("Average AUC ROC score => "+ str(sum(xgb_lst_aucroc_stratified)/len(xgb_lst_aucroc_stratified)))

Average AUC ROC score => 0.9854346619933569


## lgbm

In [607]:
print ("Average accuracy => "+ str(sum(lgbm_lst_accu_stratified)/len(lgbm_lst_accu_stratified)))

Average accuracy => 0.9929774882294404


In [608]:
print ("Average F1 score => "+ str(sum(lgbm_lst_f1_stratified)/len(lgbm_lst_f1_stratified)))

Average F1 score => 0.8768113174676309


In [609]:
print ("Average precision => "+ str(sum(lgbm_lst_prec_stratified)/len(lgbm_lst_prec_stratified)))

Average precision => 0.7949988069836411


In [610]:
print ("Average recall => "+ str(sum(lgbm_lst_recall_stratified)/len(lgbm_lst_recall_stratified)))

Average recall => 0.9779079816980172


In [611]:
print ("Average AUC ROC score => "+ str(sum(lgbm_lst_aucroc_stratified)/len(lgbm_lst_aucroc_stratified)))

Average AUC ROC score => 0.9856401396145429


## 10 folds

In [612]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

In [613]:
ada_lst_accu_stratified = []
ada_lst_f1_stratified = []
ada_lst_prec_stratified = []
ada_lst_recall_stratified = []
ada_lst_aucroc_stratified = []

xgb_lst_accu_stratified = []
xgb_lst_f1_stratified = []
xgb_lst_prec_stratified = []
xgb_lst_recall_stratified = []
xgb_lst_aucroc_stratified = []

rf_lst_accu_stratified = []
rf_lst_f1_stratified = []
rf_lst_prec_stratified = []
rf_lst_recall_stratified = []
rf_lst_aucroc_stratified = []

lgbm_lst_accu_stratified = []
lgbm_lst_f1_stratified = []
lgbm_lst_prec_stratified = []
lgbm_lst_recall_stratified = []
lgbm_lst_aucroc_stratified = []

In [614]:
xgb = XGBClassifier(scale_pos_weight=1)
ada = AdaBoostClassifier()
rf = RandomForestClassifier()
lgbm = lightgbm.LGBMClassifier()

In [615]:
    xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=1, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='gpu_hist', use_label_encoder=True,
              validate_parameters=1, verbosity=0)
    
    ada = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                           n_estimators=50, random_state=1)
    rf = RandomForestClassifier(bootstrap=True,ccp_alpha=0.0,
                               class_weight=None,criterion='gini',
                               max_depth=None,max_features='auto',
                               max_leaf_nodes=None, max_samples=None,
                               min_impurity_decrease=0.0,min_samples_leaf=1, min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=100,
                               n_jobs=-1,oob_score=False,random_state=1,verbose=0,
                               warm_start=False)
    lgbm = lightgbm.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                                  importance_type='split', learning_rate=0.1,
                                   max_depth=-1, min_child_samples=20, min_child_weight=0.001,
                                   min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
                                   objective=None, random_state=1, reg_alpha=0.0, reg_lambda=0.0,
                                   silent=True, subsample=1.0, subsample_for_bin=200000,
                                   subsample_freq=0)

In [ ]:
%%time
for train_index, test_index in skf.split(X, Y):
    print(len(test_index))
    
    x_train_fold, x_test_fold = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train_fold, y_test_fold = Y[train_index], Y[test_index]
    oversample = SMOTE()
    x_train, y_train = oversample.fit_resample(x_train_fold, y_train_fold)
    
    xgb_clf = xgb.fit(x_train,y_train)
    ada_clf = ada.fit(x_train,y_train)
    rf_clf = rf.fit(x_train,y_train)
    lgbm_clf = lgbm.fit(x_train,y_train)
    
    y_pred = ada.predict(x_test_fold)
    y_true = y_test_fold.tolist()
    ada_acc = accuracy_score(y_true,y_pred)
    ada_f1 = f1_score(y_true,y_pred)
    ada_precision = precision_score(y_true,y_pred)
    ada_recall = recall_score(y_true,y_pred)
    ada_aucroc = roc_auc_score(y_true,y_pred)
    ada_lst_accu_stratified.append(ada_acc)
    ada_lst_f1_stratified.append(ada_f1)
    ada_lst_prec_stratified.append(ada_precision)
    ada_lst_recall_stratified.append(ada_recall)
    ada_lst_aucroc_stratified.append(ada_aucroc)
    
    y_pred = xgb.predict(x_test_fold)
    xgb_acc = accuracy_score(y_true,y_pred)
    xgb_f1 = f1_score(y_true,y_pred)
    xgb_precision = precision_score(y_true,y_pred)
    xgb_recall = recall_score(y_true,y_pred)
    xgb_aucroc = roc_auc_score(y_true,y_pred)
    xgb_lst_accu_stratified.append(xgb_acc)
    xgb_lst_f1_stratified.append(xgb_f1)
    xgb_lst_prec_stratified.append(xgb_precision)
    xgb_lst_recall_stratified.append(xgb_recall)
    xgb_lst_aucroc_stratified.append(xgb_aucroc)
    
    y_pred = lgbm.predict(x_test_fold)
    lgbm_acc = accuracy_score(y_true,y_pred)
    lgbm_f1 = f1_score(y_true,y_pred)
    lgbm_precision = precision_score(y_true,y_pred)
    lgbm_recall = recall_score(y_true,y_pred)
    lgbm_aucroc = roc_auc_score(y_true,y_pred)
    lgbm_lst_accu_stratified.append(lgbm_acc)
    lgbm_lst_f1_stratified.append(lgbm_f1)
    lgbm_lst_prec_stratified.append(lgbm_precision)
    lgbm_lst_recall_stratified.append(lgbm_recall)
    lgbm_lst_aucroc_stratified.append(lgbm_aucroc)
    
    y_pred = rf.predict(x_test_fold)
    rf_acc = accuracy_score(y_true,y_pred)
    rf_f1 = f1_score(y_true,y_pred)
    rf_precision = precision_score(y_true,y_pred)
    rf_recall = recall_score(y_true,y_pred)
    rf_aucroc = roc_auc_score(y_true,y_pred)
    rf_lst_accu_stratified.append(rf_acc)
    rf_lst_f1_stratified.append(rf_f1)
    rf_lst_prec_stratified.append(rf_precision)
    rf_lst_recall_stratified.append(rf_recall)
    rf_lst_aucroc_stratified.append(rf_aucroc)
    
    if rf_f1 > rf_temp:
        rf_temp = rf_f1
        filename = str(rf_counter)+"rf_model.pkl"
        pickle.dump(rf_clf,open(filename,'wb'))
        print("rf "+str(rf_counter))
    rf_counter = rf_counter+1
        
    if ada_f1 > ada_temp:
        ada_temp = ada_f1
        filename = str(ada_counter)+"ada_model.pkl"
        pickle.dump(ada_clf,open(filename,'wb'))
        print("ada " + str(ada_counter))
    ada_counter = ada_counter+1
    
    if xgb_f1 > xgb_temp:
        xgb_temp = xgb_f1
        filename = str(xgb_counter)+"xgb_model.pkl"
        pickle.dump(xgb_clf,open(filename,'wb'))
        print("xgb "+str(xgb_counter))
    xgb_counter = xgb_counter+1
        
    if lgbm_f1 > lgbm_temp:
        lgbm_temp = lgbm_f1
        filename = str(lgbm_counter)+"lgbm_model.pkl"
        pickle.dump(lgbm_clf,open(filename,'wb'))
        print("lgbm "+str(lgbm_counter))
    lgbm_counter = lgbm_counter+1

5497


D:\anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


5497


D:\anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


rf 7
5497


D:\anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


lgbm 8
5497


D:\anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


5497


D:\anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


# rf

In [ ]:
print ("Average accuracy => "+ str(sum(rf_lst_accu_stratified)/len(rf_lst_accu_stratified)))

In [ ]:
print ("Average F1 score => "+ str(sum(rf_lst_f1_stratified)/len(rf_lst_f1_stratified)))

In [ ]:
print ("Average precision => "+ str(sum(rf_lst_prec_stratified)/len(rf_lst_prec_stratified)))

In [ ]:
print ("Average recall => "+ str(sum(rf_lst_recall_stratified)/len(rf_lst_recall_stratified)))

In [ ]:
print ("Average AUC ROC score => "+ str(sum(rf_lst_aucroc_stratified)/len(rf_lst_aucroc_stratified)))

## ada

In [ ]:
print ("Average accuracy => "+ str(sum(ada_lst_accu_stratified)/len(ada_lst_accu_stratified)))

In [ ]:
print ("Average F1 score => "+ str(sum(ada_lst_f1_stratified)/len(ada_lst_f1_stratified)))

In [ ]:
print ("Average precision => "+ str(sum(ada_lst_prec_stratified)/len(ada_lst_prec_stratified)))

In [ ]:
print ("Average recall => "+ str(sum(ada_lst_recall_stratified)/len(ada_lst_recall_stratified)))

In [ ]:
print ("Average AUC ROC score => "+ str(sum(ada_lst_aucroc_stratified)/len(ada_lst_aucroc_stratified)))

## xgb

In [ ]:
print ("Average accuracy => "+ str(sum(xgb_lst_accu_stratified)/len(xgb_lst_accu_stratified)))

In [ ]:
print ("Average F1 score => "+ str(sum(xgb_lst_f1_stratified)/len(xgb_lst_f1_stratified)))

In [ ]:
print ("Average precision => "+ str(sum(xgb_lst_prec_stratified)/len(xgb_lst_prec_stratified)))

In [ ]:
print ("Average recall => "+ str(sum(xgb_lst_recall_stratified)/len(xgb_lst_recall_stratified)))

In [ ]:
print ("Average AUC ROC score => "+ str(sum(xgb_lst_aucroc_stratified)/len(xgb_lst_aucroc_stratified)))

## lgbm

In [ ]:
print ("Average accuracy => "+ str(sum(lgbm_lst_accu_stratified)/len(lgbm_lst_accu_stratified)))

In [ ]:
print ("Average F1 score => "+ str(sum(lgbm_lst_f1_stratified)/len(lgbm_lst_f1_stratified)))

In [ ]:
print ("Average precision => "+ str(sum(lgbm_lst_prec_stratified)/len(lgbm_lst_prec_stratified)))

In [ ]:
print ("Average recall => "+ str(sum(lgbm_lst_recall_stratified)/len(lgbm_lst_recall_stratified)))

In [ ]:
print ("Average AUC ROC score => "+ str(sum(lgbm_lst_aucroc_stratified)/len(lgbm_lst_aucroc_stratified)))